# Overview
In a perfect world with perfect data, data scientists would spend all their time doing science with data. In reality, much of our time is spent *preparing* data so that we can eventually do the science part. These preparation, or cleaning steps, are necessary because datasets frequently contain messy or missing data. This lecture will cover some common practices for data preparation and cleaning. This will include:
- Introduction to Numpy
- Brief review of statistics
- Visual tools for data cleaning
- NaN handling: null labels, dropping vs imputing, imputation methods
- Noise reduction: moving average, median filtering (1D, 2D)
- If there is time: Type consistency and casting

# Numpy
Numpy is a Python package that we will use to store data and do math with that data more efficiently. We've already been introduced to numpy through `pandas`, because under the hood the values stored in a pandas DataFrame are numpy arrays. Here, we will introduce them a bit more rigorously. 

## Basics of Numpy arrays

In [ ]:
import numpy as np

The base `numpy` data type is the n-dimensional array. In a lot of ways, it behaves exactly like a list, and you can even initialize one from a list:

In [ ]:
example_list = [1, 2, 3, 4, 5]
example_array = np.array(example_list)

We can slice and extract data from a numpy array just like a list:

In [ ]:
example_array[0]

In [ ]:
example_array[-2:]

And we can apply Boolean logic to arrays just like we do on `pandas` Series objects:

In [ ]:
example_array < 3

This helps us extract portions of arrays based on logical conditions:

In [ ]:
example_array[example_array < 3]

Numpy handles NaN values using its built-in `np.nan` type. Here we can "nan-out" particular values in the array:

In [ ]:
example_array[(example_array % 2 == 0)] = np.nan

Whoops! that didn't quite work because of a type mismatch. The array was initialized with integer values, and we tried to replace some values with `np.nan`, which is a float. One of the situations where the difference matters.

In [ ]:
example_array = np.array(example_list, dtype=np.float64) # Could also do example_array = example_array.astype(float)

In [ ]:
example_array[(example_array % 2 == 0)] = np.nan

In [ ]:
example_array

Numpy arrays can also be multi-dimensional, like a DataFrame that has values along the rows and columns. We call this a matrix:

In [ ]:
example_matrix = np.random.rand(3, 3)  # Initialize a random 3 x 3 matrix

In [ ]:
example_matrix

We can extract row and column values in a similar way to the 1d case:

In [ ]:
col_1 = example_matrix[:, 0]
col_1

In [ ]:
row_1 = example_matrix[0, :]
row_1

In [ ]:
last_element = example_matrix[2, 2]
last_element

## Initialization shortcuts

If you want an array of linearly spaced values between two endpoints:

In [ ]:
np.linspace(0.1, 1, 10)

If you want an array of values with a certain spacing between them:

In [ ]:
np.arange(1, 10, 0.5)

If you want an array full of zeros or ones of a certain shape:

In [ ]:
np.zeros((3, 3))

In [ ]:
np.ones((5,))

Or an array of NaN:

In [ ]:
np.zeros((5, 2)) * np.nan

## Math with arrays
The real power of numpy arrays is the math that it lets us do. We'll learn about all the linear algebra operations next lecture, but for now the most important thing to understand are the elementwise operations that lists don't allow.

This isn't math, it's concatenation:

In [ ]:
a_list = [1, 2, 3]
b_list = [4, 5, 6]
a_list + b_list

And this doesn't even work:

In [ ]:
a_list * b_list

Nor this:

In [ ]:
a_list ** 2

But with a numpy array we can do elementwise math:

In [ ]:
a_array = np.array([1, 2, 3])
b_array = np.array([4, 5, 6])
a_array + b_array

In [ ]:
a_array * b_array

In [ ]:
a_array ** 2

In [ ]:
b_array + 5

In [ ]:
2 * b_array

# Quick Breakout: Numpy Practice
1. Create a numpy array `x` that contains 100 values evenly spaced between 10 and 20
2. Create another numpy array `y`, where each element $y_i$ is given by $y_i = 2 x_i + 5$ for $i = 1, \dots, 100$.
3. Use matplotlib to plot $x$ vs $y$

# Stats Review
In data science we often want to determine the probability that a certain data point (or group of data points) in a dataset are real (i.e., to be trusted), vs noise (not to be trusted). Noisy data can have many sources. For example, bad sensors, bad data entry, wrong units, and general upstream bugs in data processing can all negatively impact data quality, and it is often our job to identify *when* that is the case. Statistics can help answer the question: how likely (or unlikely) is it to observe a particular outcome? Here we will review some basic statistics concepts for that purpose. 

## Mean, Median, Standard Deviation

The mean value of a vector $x$ of length $n$ is defined as
$$ \overline{x} = \frac{1}{n} \sum_{i = 1}^n x_i $$
The mean is a *measure of central tendency*, meaning that it gives us an estimate of a typical value of a dataset. 

In [ ]:
x = np.array([1, 2, 3, 4, 5, 100, 200])
x_bar = np.mean(x)
print(x_bar)

Another common measure of central tendency is the median, which gives us the 50th percentile value of a vector. You'll notice that the median is not as influenced by outliers as the mean is:

In [ ]:
x_med = np.median(x)
print(x_med)

While the mean and median provide typical values of a variable, the standard deviation tells us how much variability there is in the variable. In other words, how *far away from the mean* are typical values of the variable? Standard deviation is defined as
$$ \sigma = \sqrt{\frac{\sum_{i = 1}^n (x_i - \overline{x})^2}{n}},$$
though you will sometimes see $n-1$ instead of $n$ in the denominator. For a large enough sample size, this barely makes a difference.  For what it's worth, `numpy` uses $n$:

In [ ]:
x_std = np.std(x)
print(x_std)

Note: If our array contains NaN values, we can use the functions `np.nanmean`, `np.nanmedian`, and `np.nanstd` instead, which ignores NaN values when calculating the statistics. 

## Probability density functions
A dataset can be roughly summarized with statistics like the mean, median, and standard deviation. But a much more holistic view can be obtained by the probability density function for a variable or a process. 

A function $f(x)$ is considered a probability density function if:
- $f(x) \geq 0$ over the function's support
- $\int_{-\infty}^{\infty} f(x)\text{d}x = 1$
- $\int_{a}^{b} f(x) \text{d}x = P(a \leq x \leq b)$

That final bullet point is the most important, and says: the probability that $x$ is between $a$ and $b$ is given by the integral of the probability density function between $a$ and $b$.

The takeaway is that if we know the probability density function for a certain event or process, we are in good shape because we can calculate probabilities based on it via integration. 

We can also calculate statistics like the mean, median, and standard deviation from a PDF. The mean $\mu$ is defined as:
$$ \mu = \int_{-\infty}^{\infty} x f(x) dx$$

And the median $\tilde{\mu}$ is given by:
$$\tilde{\mu} = \int_{-\infty}^{\tilde{\mu}} f(x) dx = 0.5$$

And the standard deviation $\sigma$:
$$\sigma = \sqrt{\int_{-\infty}^{\infty} (x - \mu)^2 f(x) dx}$$

Though in practice, the integration bounds may be finite and restricted to the domain of the specific PDF you are integrating. 

## Probability distributions
In general, we won't know the probability density function for whatever we are interested in. But there are certain general *probability distributions* that statisticians have found are widely applicable to a range of problems. We'll talk about a couple of the most common ones.

### Normal Distribution
The normal (or Gaussian) distribution is described by
$$ f(x) = \frac{1}{\sqrt{2\pi} \sigma} \exp\left(-\frac{(x - \mu)^2}{2 \sigma^2}\right) $$
where $\mu$ is the mean of the distribution and $\sigma$ is the standard deviation. We can plot this in python:

In [ ]:
import matplotlib.pyplot as plt
x = np.linspace(-10, 10, 100)
mu = 3
sigma = 2
density = np.exp(-(x - mu)**2 / (2 * sigma**2)) / np.sqrt(2 * np.pi * sigma)
plt.figure(figsize=(4, 3))
plt.plot(x, density)
plt.xlabel("X")
plt.ylabel("Density")

### Quick breakout: try changing the mean and standard deviation of the distribution, $\mu$ and $\sigma$. How does this affect the distribution? 

What the plots of the normal distribution tell us is that values around the mean $\mu$ are the most common, because there is more area under the curve around that value. Compare that to $x = -10$ (for $\mu = 3$), there is effectively zero area under the curve, so if a variable in our dataset was described by this distribution and we saw a value of $-10$, that would be highly suspect. However, if we increased the standard deviation significantly, then the density would spread out, and values far from the mean would become increasingly likely. 

There are some simple rules of thumb that you should memorize for a normal distribution:
- 68% of the area under the curve is within 1 standard deviation of the mean
- 95% of the area under the curve is within 2 standard deviations of the mean
- 99.7% of the area under the curve is within 3 standard deviations of the mean

In [ ]:
from PIL import Image
Image.open("normal_dist_rule.jpg")

How do we know if a normal distribution is appropriate to use for our data? Usually, we just make a histogram and see how normal it looks. Let's look at a long-term oceanographic dataset from the north pacific to see what the distributions look like for different variables

In [ ]:
import pandas as pd
df = pd.read_csv("https://github.com/galenegan/DATA-3320/raw/main/climate/north_pacific.csv")
df = df.rename(columns={"sst": "sea_surface_temperature", "u10": "wind_velocity", "hsig": "wave_height"})

In [ ]:
fig = plt.figure()
df.hist(column=["sea_surface_temperature", "wind_velocity", "wave_height"], bins="auto", figsize=(10,8))

How would we categorize each of the variables?
- Sea surface temperature: bimodal (why?). Subsets may be normal
- Wind velocity: pretty normal, slightly skewed
- wave height: not normal, right-skewed, seems to be clamped at zero

Because wind velocity is approximately Gaussian, we can use the properties of the normal distribution to classify different ranges of values. For example, if someone asked you to identify the range of wind velocity that you might encounter 95% of the time, you could simply do:

In [ ]:
wind_range_lower_bound = df["wind_velocity"].mean() - 2 * df["wind_velocity"].std()
wind_range_upper_bound = df["wind_velocity"].mean() + 2 * df["wind_velocity"].std()
(wind_range_lower_bound, wind_range_upper_bound)

# Visual tools for outlier detection

Let's say someone hands you a dataset and tells you to clean it up by getting rid of suspicious outlier values. We'll use a subset of the [UCI Wine Dataset](https://archive.ics.uci.edu/dataset/109/wine) as an example. Contained in the data are the alcohol and proline contents of 2 different types of wine. However, data from a third and pretty different type snuck in as well, and we want to identify which data points are from that third variety. 

In [ ]:
df_full = pd.read_csv("wine.csv")
df = df_full.drop(columns=["outlier"])

The best way to start is with a pairplot:

In [ ]:
import seaborn as sns
fig = sns.pairplot(df, height=3, aspect=1.25)

Do any clumps of data points look like they might not fit in with the rest? 

Let's check by highlighting the outlier values from the full dataframe:

In [ ]:
fig = sns.pairplot(df_full, hue="outlier", height=3, aspect=1.25)

So it was indeed the high alcohol + high proline wines that didn't belong. Let's see if we could have done a similar outlier identification by normalizing the proline data and excluding values outside a certain number of standard deviations from the mean. 

Here, we use a common trick to make right-skewed data look more Normal: take its natural logarithm

In [ ]:
df["log_proline"] = np.log(df["proline"])
plt.hist(df["log_proline"], bins="auto")

In [ ]:
n_std_cutoff = 1.5
upper_bound = df["log_proline"].mean() + n_std_cutoff * df["log_proline"].std()

In [ ]:
upper_bound

In [ ]:
df["estimated_outlier"] = (df["log_proline"] >= upper_bound)
fig = sns.pairplot(df, vars=["alcohol", "proline"], hue="estimated_outlier", height=3, aspect=1.25)

# NaN Handling
Sometimes, for whatever reason, individual data points will be missing. They are usually filled in with some variation of `NaN`, which stands for `Not A Number`, and is an example of a [sentinel value](https://en.wikipedia.org/wiki/Sentinel_value). When we encounter NaN values, we have choices to make about how to handle them in our analysis. And when we are creating a dataset, we have choices to make about how to label NaN values. We'll start with that first. 

## Proper labeling
There is no single right way to label a `NaN` value, because the right way varies by situation. But here are some general guidelines you can follow.
- If you need to label missing values in a numpy array or Pandas dataframe, `np.nan` is a good approach. For example, `df.loc[bad_indices, "column_name"] = np.nan`
- If you are labeling missing values in a Python dictionary or json blob that could potentialy end up on the internet or get passed through an internal API, then Python's built-in `None` is the best option. This is because the json format can't handle `np.nan`, but `None` will automatically get converted to the Java `null`.
- If you are restricted to using numeric values and cannot label something `NaN` or `None`, then it is ok to define a numeric sentinel value, ideally something very obviously out-of-place in the dataset. Using something like `-999` or `-1` for variables that are always nonnegative is common.
- As an example of what not to do: I once had to deal with data from a customer who labeled `NaN` data for vessel bearing (the direction a vessel is traveling) as `bearing = 0`. This is *insane*, because a bearing of 0 degrees conventionally means that you are heading north. Confusion ensued! Accusations were made! Relationships soured! And all because of improper NaN labeling.

## Can I just drop NaN values?
It is often tempting to just drop any rows of a dataset where one or more of the features is NaN. This can be ok if:
- Your dataset is large enough already, and dropping the NaN rows won't reduce its size by more than a few %
- The data that you are dropping isn't special in some way. In other words, the other features (that aren't NaN) have a similar distribution both within and outside of the dropped portion.

But if, by dropping NaNs, you are getting rid of a significant portion of your data, then you should probably try to impute those missing values. 

## Imputation
Imputation refers to the process of replacing a `NaN` value in a dataset with an educated guess for a real value that might take its place. We'll learn about a couple ways to do this. 

### Interpolation
Interpolation involves filling in data gaps by using data that came from before and after the gap. Let's go back to our sea surface temperature data and see an example:

In [ ]:
df = pd.read_csv("https://github.com/galenegan/DATA-3320/raw/main/climate/north_pacific.csv")
df = df.rename(columns={"sst": "sea_surface_temperature", "u10": "wind_speed", "hsig": "wave_height"})
sst = df.loc[:100, "sea_surface_temperature"]

In [ ]:
import matplotlib.pyplot as plt
plt.plot(sst)

Now let's replace a bunch of random values with NaN:

In [ ]:
nan_pct = 0.2 
N = len(sst) 
index_to_nan = np.random.choice(np.arange(N), int(nan_pct * N))
sst

In [ ]:
sst[index_to_nan] = np.nan

In [ ]:
plt.plot(sst)

Linear Interpolation

In [ ]:
sst_linear_interp = sst.interpolate(method="linear")
plt.plot(sst_linear_interp, label="linearly interpolated")
plt.plot(sst, label="original")
plt.legend()

Cubic Interpolation

In [ ]:
sst_cubic_interp = sst.interpolate(method="cubic")
plt.plot(sst_cubic_interp, label="cubically interpolated")
plt.plot(sst, label="original")
plt.legend()

Nearest Neighbor Interpolation

In [ ]:
sst_nearest_interp = sst.interpolate(method="nearest")
plt.plot(sst_nearest_interp, label="Nearest neighbor interpolated")
plt.plot(sst, label="original")
plt.legend()

### Advanced Imputers
Sometimes you have more information available than just a single variable with gaps in it. In those cases, it can be worth using a more advanced imputation technique that takes other features in your dataset into consideration. We'll use a sample dataset to highlight this, where `x1` and `x2` are features, and `y` is the thing we are trying to predict. 

In [ ]:
df = pd.DataFrame({'x1': [2.1, 3.2, 4.1, 1.15, 5.05, 6.1, 7.2, 8.1, np.nan],
                   'x2': [2.9, np.nan, -0.9, 3.3, -2.9, -2.6, -4.4, -6.4, -1.7],
                   'y': [6.45,  9.6 , 10.4 ,  5.9, 12.6 , 16.33, 18.6 , 20.3, 19.5]})

df

Plotting the data

In [ ]:
plt.figure(figsize = (6,5))

plt.plot(df['x1'], df['x2'], 'o')

plt.xlabel('x1', fontsize = 14)
plt.ylabel('x2', fontsize = 14)
plt.tick_params(labelsize = 12)

First we pre-process the data to remove its mean and normalize for unit variance. This is a standard step in almost all machine learning applications

In [ ]:
from sklearn import preprocessing
from sklearn.impute import KNNImputer
scaler = preprocessing.StandardScaler().fit(df)
df_scaled = pd.DataFrame(scaler.transform(df), columns = ['x1', 'x2', 'y'])
df_scaled

Next we rely on the K-nearest-neighbors imputer to impute the missing values. Another good choice is the IterativeImputer, which relies on a different algorithm under the hood but has the same implementation. 

In [ ]:
imputer = KNNImputer(n_neighbors=2)

In [ ]:
X = df_scaled[['x1', 'x2']]

In [ ]:
df_knn_scaled = df_scaled.copy()

In [ ]:
df_knn_scaled[['x1', 'x2']] = imputer.fit_transform(X)

Here is our imputed dataframe in the scaled coordinates

In [ ]:
df_knn_scaled

And here we do the `inverse_transform` to get the data back in its original units/coordinates

In [ ]:
df_knn = pd.DataFrame(scaler.inverse_transform(df_knn_scaled), columns = ['x1', 'x2', 'y'])

In [ ]:
df_knn

We can plot it to make sure everything makes sense

In [ ]:
plt.figure(figsize = (6,5))

plt.plot(df_knn['x1'], df_knn['x2'], 'o', label = 'imputed')

plt.plot(df['x1'], df['x2'], 'o', label = 'original')

plt.xlabel('x1', fontsize = 14)
plt.ylabel('x2', fontsize = 14)
plt.legend()
plt.tick_params(labelsize = 12)

# Smoothing Techniques
Finally, we will discuss smoothing techniques. Usually due to bad sensors, some data are just kind of... bad. Every few observations will be much higher or lower than it ought to be. Smoothing techniques help to mellow things out, removing the noise and providing a better representation of the "real" signal. These techniques will be demonstrated on a sine wave `y` with synthetically added noise:

In [ ]:
x = np.linspace(0, 2 * np.pi, 200)
noise = np.random.randn(len(x)) * 0.75
y = np.sin(x) + noise
plt.plot(y, label="raw signal y", alpha=0.6)
plt.legend()

## Moving Average
A moving average places a window along a signal, taking an average at that window position, and then slides the window over by one place, takes another average, and keeps going until it gets to the end of the signal. This is a nice way to smooth-out variability in a signal. 

In [ ]:
from scipy.ndimage import uniform_filter1d
yfilt = uniform_filter1d(y, size=5)  # The size parameter gives the width of the window that the average is taken over
plt.plot(y, label="y", alpha=0.6)
plt.plot(yfilt, label="yfilt", linewidth=2)
plt.legend()

## Quick breakout: try modifying the size parameter in the moving average. How does it change the filtered signal?

## Median filter
A median filter is nearly identical to a moving average, except it takes the median over a window instead of the mean. This often results in better noise reduction, because median is more resilient to outliers than the mean. 

In [ ]:
from scipy.ndimage import median_filter
yfilt = median_filter(y, size=15)  # The size parameter gives the width of the window that the average is taken over
plt.plot(y, label="y", alpha=0.6)
plt.plot(yfilt, label="yfilt", linewidth=2)
plt.legend()

The median filter is often used in multiple dimensions as well, particularly in image processing applications. Let's see how we can smooth this image:

In [ ]:
from PIL import Image
from io import BytesIO
import requests

In [ ]:
url = "https://github.com/galenegan/DATA-3320/blob/main/classification_data/cat.jpeg?raw=true"
response = requests.get(url)
img = Image.open(BytesIO(response.content))

In [ ]:
img_array = np.array(img)
img_filt = median_filter(img_array, size=(15, 15, 1))

In [ ]:
plt.imshow(img_array)

In [ ]:
plt.imshow(img_filt)

# Appendix

##  Type consistency and casting
One of the most common problems you'll find with a dataset is data stored as the wrong type, e.g., strings instead of floats, floats instead of ints, etc. Before analyzing data quantitatively, it is important to make sure that all your types are correct. We'll use a modified version of the wine dataset to highlight some common problems and how to fix them.

In [ ]:
df = pd.read_csv("wine_types.csv")

In [ ]:
df.head()

In [ ]:
df.info()

Each of the columns has at leaset one problem with it. Can you name them?
1. Alcohol:
2. Proline:
3. Outlier:

Let's fix the outlier column first, because it's the easiest:

In [ ]:
df["outlier"] = df["outlier"].astype(int)

or...

In [ ]:
df["outlier"] = df["outlier"].astype(bool)

In [ ]:
df["outlier"]

And now alcohol:

In [ ]:
df["alcohol"] = df["alcohol"].str.replace(",", ".").astype(float)

In [ ]:
df["alcohol"]

And proline:

In [ ]:
df["proline"] = [float(i.split("= ")[-1]) for i in df["proline"].values]

In [ ]:
df["proline"]

## Log-normal distribution
Log-normal distributions arise in lots of physical scenarios where a variable is restricted to positive values. If a variable is log-normally distributed, that means that the natural log of the variable is normally-distributed. Let's look at wave height to see whether it might be log-normally distributed

In [ ]:
df["log_wave_height"] = np.log(df["wave_height"])
fig = plt.figure()
plt.hist(df["log_wave_height"], bins="auto")

Still a little skewed, but much better! Now we can more easily apply our normal distribution rules of thumb to any given value of wave height, as long as we log-transform it first. For example, we might want to do some analysis that is valid for 95% most common wave conditions. We could filter our dataframe using the 95% rule:

In [ ]:
lower_bound = df["log_wave_height"].mean() - 2 * df["log_wave_height"].std()
upper_bound = df["log_wave_height"].mean() + 2 * df["log_wave_height"].std()
condition = ((df["log_wave_height"] >= lower_bound) & (df["log_wave_height"] <= upper_bound))
df_filt = df.loc[condition]

In [ ]:
plt.hist(df_filt["log_wave_height"], bins="auto")

And plotting again in the original un-transformed space:

In [ ]:
plt.hist(df_filt["wave_height"], bins="auto")

## Breakout: Outlier identification
Load the seismic dataset contained in `seismic.arff`. About 93% of the dataset contains seismic readings that were not correlated to an eventual earthquake (`class = 0`). But the remaining rows contain data that did result in an earthquake (`class = 1`). Using visual outlier detection tools, can you isolate the rows associated with `class = 1` based on the other measured features?